In [1]:
# テキストデータの準備
import os
import json

# コーパスのディレクトリを設定
file_path = './projectnextnlp-chat-dialogue-corpus/json/rest1046/'
# ファイルの一覧を取得
file_dir = os.listdir(file_path)
# 正解ラベルとテキストデータを保持するリスト
label_text = []

# ファイルごとに対話データを整形する
for file in file_dir:
    # JSONファイルの読み込み
    r = open(file_path + file, 'r', encoding='utf-8')
    json_data = json.load(r)
        
    # 発話データ配列から発話テキストと破綻かどうかの正解データを抽出
    for turn in json_data['turns']:
        turn_index = turn['turn-index'] # turn-indexキー(対話のインデックス)
        speaker = turn['speaker']       # speakerキー("U"人間、"S"システム)
        utterance = turn['utterance']   # utteranceキー(発話テキスト)
        
        # 先頭行(システムの冒頭の発話)以外を処理
        if turn_index != 0:
            # 人間の発話（質問）のテキストを抽出
            if speaker == 'U':
                #u_text = ''
                u_text = utterance

            # システムの回答内容が破綻かどうかを抽出
            else:
                a = ''
                sys = turn['utterance'] # システムの発話（応答）を抽出
                t = turn['annotations'][0] # １つ目のアノテーションを抽出                
                a = t['breakdown']      # アノテーションのフラグを抽出
                if a == 'O':            # O（破綻していない）を0で置換
                    label = 0
                else:                   # O以外は破綻とし1で置換
                    label = 1
                # 正解ラベルをタブで区切って「人間の発話#システムの発話」を連結
                tmp1 = str(label) + '\t' + u_text + ' # ' +sys
                # タブで区切ってリストにする
                tmp2 = tmp1.split('\t')
                # 作成したリストを要素としてlabel_textに追加
                label_text.append(tmp2)

In [2]:
# 形態素への分解と正解ラベルのリストの生成
from janome.tokenizer import Tokenizer # janomeのパッケージをインポート
import re                              # 正規表現ライブラリ

t = Tokenizer()                       # Tokenizerクラスのオブジェクトを生成
separation_tmp = []                   # 形態素を一時保存するリスト

# 形態素に分解
for row in label_text:
    # リストから発話テキストの部分を抽出して形態素解析を実行
    tokens = t.tokenize(row[1])
    # 形態素の見出しの部分を取得してseparation_tmpに追加
    separation_tmp.append(
        [token.surface for token in tokens if (
            not re.match('記号', token.part_of_speech)             # 記号を除外
            and (not re.match('助詞', token.part_of_speech))       # 助詞を除外
            and (not re.match('助動詞', token.part_of_speech))     # 助動詞を除外
            and (not re.match('動詞,非自立', token.part_of_speech))# 動詞,非自立を除外
            and (not re.match('名詞,非自立', token.part_of_speech))# 名詞,非自立を除外
            )
         ])
    # 空の要素があれば取り除く
    while separation_tmp.count('') > 0:
        separation_tmp.remove('')

# 正解ラベルをint型に変換してリストに格納
train_y_tmp = [int(label[0]) for label in label_text]

separation = [] # 形態素のリスト
train_y = []    # 正解ラベルのリスト

# separation_tmpの値が存在すれば、
# 形態素のリストと正解ラベルのリストを作成
for x, y in zip(separation_tmp, train_y_tmp):
    if x:
        separation.append(x)
        train_y.append(y)

In [3]:
# 単語の出現回数を記録して辞書を作成　
from collections import Counter        # カウント処理のためのライブラリ
import itertools                       # イテレーションのためのライブラリ
# {単語：出現回数}の辞書を作成
word_frequency = Counter(itertools.chain(* separation))

In [4]:
# 単語を頻度降順に並べ替え
word_list = []
# most_common()で出現回数順に要素を取得しword_listに追加
for w in word_frequency.most_common():
    word_list.append(w[0])
    
# 頻度順に並べた単語をキーに、1から始まる連番を値に設定
word_dic = {}
for i, word in enumerate(word_list, start=1):
    word_dic.update({word: i})

In [5]:
# 単語を出現頻度の数値に置き替える
train_x = [[ word_dic[word] for word in sp] for sp in separation ]

In [6]:
# TensorFlowライブラリ
import tensorflow as tf
# TFLearnライブラリ
import tflearn
# データの前処理を行うライブラリ
from tflearn.data_utils import to_categorical, pad_sequences

# 単語データの配列のサイズを揃える
trainX = pad_sequences(train_x, maxlen=32, value=0.)
# # 正解ラベルをOne-hot表現にする
trainY = to_categorical(train_y, nb_classes=2)

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [7]:
# RNNの構築

# 初期化
tf.reset_default_graph()

## 入力層の作成
net = tflearn.input_data([None, 32])

## 中間層の作成
# 単語埋め込み層
net = tflearn.embedding(
    net,
    input_dim=len(word_dic) + 1, # 単語の総数に0のための1を加算,
    output_dim=128
)

# LSTMブロック
net = tflearn.lstm(net, 128, dropout=0.5, return_seq=True)
net = tflearn.lstm(net, 128, dropout=0.5, return_seq=True)
net = tflearn.lstm(net, 128, dropout=0.5, return_seq=False)

## 出力層の作成 
net = tflearn.fully_connected(net, 2, activation='softmax')

# 学習条件の設定
net = tflearn.regression(net,
                         optimizer='adam',
                         learning_rate=0.001,
                         loss='categorical_crossentropy'
                        )

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
# 学習の実行
model = tflearn.DNN(net)
model.fit(trainX,
          trainY,
          n_epoch=50,
          batch_size=32,
          validation_set=0.2,
          shuffle=True,
          show_metric=True
         )

Training Step: 13099  | total loss: 0.64341 | time: 23.016s
| Adam | epoch: 050 | loss: 0.64341 - acc: 0.6676 -- iter: 8352/8368
Training Step: 13100  | total loss: 0.64006 | time: 24.511s
| Adam | epoch: 050 | loss: 0.64006 - acc: 0.6727 | val_loss: 0.69171 - val_acc: 0.5875 -- iter: 8368/8368
--
